<a href="https://colab.research.google.com/github/BhuvanKalyanGV/Web_scapping/blob/main/pdfToDataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tabula-py pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 87.5 MB/s eta 0:00:00


In [ ]:
pip install camelot-py

In [ ]:
import requests
import tempfile
import os
import camelot
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# Download the PDF file
url = "https://download.terna.it/terna/Monthly%20Report%20on%20the%20Electricity%20System_December_23_EN_8dc1dc5bc60e6a5.pdf"
response = requests.get(url)

# Save the PDF to a temporary file
with tempfile.TemporaryDirectory() as tmp_dir:
    pdf_path = os.path.join(tmp_dir, "temp.pdf")
    with open(pdf_path, "wb") as f:
        f.write(response.content)

    # Extract tables from specific pages (9) using Camelot
    tables = camelot.read_pdf(pdf_path, pages='9', flavor='stream')

    # Create a new Excel file
    excel_file = "Monthly Report on the Electricity System December 2023.xlsx"
    writer = pd.ExcelWriter(excel_file, engine='openpyxl')

    # Define the desired sheet names
    sheet_names = ["2023 Historical Monthly Energy Balance Sheet", "2022 Historical Monthly Energy Balance Sheet"]

    # Loop through the extracted tables and write them to respective sheets
    for idx, table in enumerate(tables):
        # Extract table headers
        headers = table.df.columns.tolist()

        # Use the predefined sheet name
        sheet_name = sheet_names[idx]

        # Write the table to the specified sheet, excluding the first row
        table.df.iloc[1:].to_excel(writer, sheet_name=sheet_name, index=False, header=False)

        # Load the sheet and bold the first row (column names)
        sheet = writer.sheets[sheet_name]
        for cell in sheet[1]:
            cell.font = Font(bold=True)

    # Save the Excel file
    if not writer.book.worksheets:
        # Create a default sheet if no sheets are present
        writer.book.create_sheet()
    writer.book.save(excel_file)